In [ ]:

from numpy import *
set_printoptions(legacy = '1.25')

u = array([1,2,3,4,5])
v = array([6,7,8,9,10])

dot(u,v), 1*6 + 2*7 + 3*8 + 4*9 + 5*10


In [ ]:

dot(u,v) == sum(u*v)


In [ ]:

# length of v

sqrt(dot(v,v))


In [ ]:

w = array([11,12,13,14,15])

A = column_stack([u,v,w])
a = array([1,6,11])

dot(A,a) == array([ dot(row,a) for row in A ])


In [ ]:
 
dot(A,a) == sum(A*a, axis = 1)


In [ ]:

B = A.T
dot(A,B) == array([ sum(A*b, axis = 1) for b in B.T ])


In [ ]:

dot(u, dot(A,a)) == dot(dot(A.T,u), a)


In [ ]:

dot(A,B).T == dot(B.T,A.T)


In [ ]:

from sklearn import datasets
	
iris = datasets.load_iris()
dataset = iris['data']
	
# dataset is Nxd
mu = mean(dataset, axis = 0)
A = dataset - mu
	
N = len(dataset)
dot(A.T, A) / N


In [ ]:

Q = cov(dataset.T, bias = True)
	
mu, Q, trace(Q)


In [ ]:

from numpy import *
from scipy.linalg import inv
from sklearn.preprocessing import StandardScaler

# standardize dataset
standardized = StandardScaler().fit_transform(dataset)

Qcorr = corrcoef(dataset.T)
Qcov = cov(standardized.T, bias = True)

Sigma = diag(sqrt(diag(Q)))
Qprime = dot(inv(Sigma), dot(Q, inv(Sigma)))

Sigma, Qcov, Qcorr, Qprime


In [ ]:

shapes = (3,1,2), (7,2), (5,3,7,1)
u = broadcast_shapes(*shapes)
u == (5,3,7,2)


In [ ]:

from numpy.random import default_rng as rng
samples = rng().random

arrays = tuple( samples(s) for s in shapes )
all( shape(a) == u for a in broadcast_arrays(*arrays) )


In [ ]:

# coding broadcast_shapes from scratch

def bs(*shapes):
	d = max([ len(s) for s in shapes ])
	# max over axes in reverse order
	n = tuple( max([ s[-(d-i)] for s in shapes if d-i <= len(s) ]) for i in range(d) )
	for s in shapes:
		for i in range(d):
			if i < len(s) and s[-i-1] != 1 and s[-i-1] != n[-i-1]:
				raise ValueError('Shapes cannot be broadcast to a single shape.')
	return n

bs(*shapes)


In [ ]:

# broadcast array a to shape n

def broadcast(a, n):
	for i in range(len(n)):
	# proceed from last axis to first axis
		s = shape(a)
		if i >= len(s): 
			a = array([ a for _ in range(n[-i-1]) ])
		elif s[-i-1] == n[-i-1]: pass
		elif s[-i-1] == 1:
			a = moveaxis(a, -i-1, 0)
			a = array([ a[0] for _ in range(n[-i-1]) ])
			a = moveaxis(a, 0, -i-1)
		else: 
			raise ValueError('Array cannot be broadcast to shape.') 
	return a


In [ ]:

# coding broadcast_arrays from scratch

def ba(*arrays):
	shapes = tuple( shape(a) for a in arrays )
	n = bs(*shapes)
	return tuple( broadcast(a, n) for a in arrays )

all( a == b for a, b in zip(broadcast_arrays(*arrays), ba(*arrays)) )  


In [ ]:

def row(i,d): return [ (-1)**(i+j) for j in range(d) ]
def R(d): return array([ row(i,d) for i in range(d) ])
